# Labelme转mask-批量

同济子豪兄 2023-6-8

## 查看数据集目录结构

In [9]:
import seedir as sd
sd.seedir('Watermelon87_Semantic_Seg_Labelme', style='emoji', depthlimit=1)

ModuleNotFoundError: No module named 'seedir'

## 进入数据集目录

In [4]:
import os
import json
import numpy as np
import cv2
import shutil

from tqdm import tqdm

## 数据集及类别信息

In [11]:
os.getcwd()

'F:\\work\\python\\clone\\utils\\Label2Everything\\labelme2mask'

In [12]:
Dataset_Path = r'F:\work\python\clone\dataset\rebar2d' # 数据集路径    

## 每个类别的信息及画mask的顺序（按照由大到小，由粗到精的顺序）

In [13]:
# 0-背景，从 1 开始
class_info = [
    {'label':'rebar', 'type':'polygon', 'color':1},                    # polygon 多段线

]

In [14]:
img_bgr = cv2.imread(os.path.join(Dataset_Path, 'images', 'Image_20240622152006003.jpg'))

print(os.path.join(Dataset_Path, 'images', 'Image_20240622152006003.jpg'))
img_mask = np.zeros(img_bgr.shape[:2]) # 创建空白图像 0-背景

print(img_bgr.shape)
img_mask2=cv2.imread(r"F:\work\python\clone\dataset\rebar2d\mask\Image_20240622152540884.png")
print(img_mask2.shape)

F:\work\python\clone\dataset\rebar2d\images\Image_20240622152006003.jpg
(2048, 3072, 3)
(2048, 3072, 3)


## 单张图像labelme转mask函数

In [1]:
def labelme2mask_single_img(img_path, labelme_json_path):
    '''
    输入原始图像路径和labelme标注路径，输出 mask
    '''
    
    img_bgr = cv2.imread(img_path)
    img_mask = np.zeros(img_bgr.shape[:2]) # 创建空白图像 0-背景
    
    with open(labelme_json_path, 'r', encoding='utf-8') as f:
        labelme = json.load(f)
        
    for one_class in class_info: # 按顺序遍历每一个类别
        for each in labelme['shapes']: # 遍历所有标注，找到属于当前类别的标注
            if each['label'] == one_class['label']:
                if one_class['type'] == 'polygon': # polygon 多段线标注

                    # 获取点的坐标
                    points = [np.array(each['points'], dtype=np.int32).reshape((-1, 1, 2))]

                    # 在空白图上画 mask（闭合区域）
                    img_mask = cv2.fillPoly(img_mask, points, color=one_class['color'])

                elif one_class['type'] == 'line' or one_class['type'] == 'linestrip': # line 或者 linestrip 线段标注

                    # 获取点的坐标
                    points = [np.array(each['points'], dtype=np.int32).reshape((-1, 1, 2))]

                    # 在空白图上画 mask（非闭合区域）
                    img_mask = cv2.polylines(img_mask, points, isClosed=False, color=one_class['color'], thickness=one_class['thickness']) 

                elif one_class['type'] == 'circle': # circle 圆形标注

                    points = np.array(each['points'], dtype=np.int32)

                    center_x, center_y = points[0][0], points[0][1] # 圆心点坐标

                    edge_x, edge_y = points[1][0], points[1][1]     # 圆周点坐标

                    radius = np.linalg.norm(np.array([center_x, center_y] - np.array([edge_x, edge_y]))).astype('uint32') # 半径

                    img_mask = cv2.circle(img_mask, (center_x, center_y), radius, one_class['color'], one_class['thickness'])

                else:
                    print('未知标注类型', one_class['type'])
                    
    return img_mask

In [5]:
import cv2
import numpy as np
import os
import json
from PIL import Image

def labelme2mask_single_img(img_path, labelme_json_path):
    '''
    输入原始图像路径和labelme标注路径，输出单通道灰度 mask
    '''
    
    img_bgr = cv2.imread(img_path)
    img_mask = np.zeros(img_bgr.shape[:2], dtype=np.uint8) # 创建单通道空白图像
    
    with open(labelme_json_path, 'r', encoding='utf-8') as f:
        labelme = json.load(f)
        
    for one_class in class_info: # 按顺序遍历每一个类别
        for each in labelme['shapes']: # 遍历所有标注，找到属于当前类别的标注
            if each['label'] == one_class['label']:
                if one_class['type'] == 'polygon': # polygon 多段线标注

                    # 获取点的坐标
                    points = [np.array(each['points'], dtype=np.int32).reshape((-1, 1, 2))]

                    # 在空白图上画 mask（闭合区域）
                    img_mask = cv2.fillPoly(img_mask, points, color=one_class['color'])

                elif one_class['type'] == 'line' or one_class['type'] == 'linestrip': # line 或者 linestrip 线段标注

                    # 获取点的坐标
                    points = [np.array(each['points'], dtype=np.int32).reshape((-1, 1, 2))]

                    # 在空白图上画 mask（非闭合区域）
                    img_mask = cv2.polylines(img_mask, points, isClosed=False, color=one_class['color'], thickness=one_class['thickness']) 

                elif one_class['type'] == 'circle': # circle 圆形标注

                    points = np.array(each['points'], dtype=np.int32)

                    center_x, center_y = points[0][0], points[0][1] # 圆心点坐标

                    edge_x, edge_y = points[1][0], points[1][1]     # 圆周点坐标

                    radius = np.linalg.norm(np.array([center_x, center_y] - np.array([edge_x, edge_y]))).astype('uint32') # 半径

                    img_mask = cv2.circle(img_mask, (center_x, center_y), radius, one_class['color'], one_class['thickness'])

                else:
                    print('未知标注类型', one_class['type'])
                    
    return img_mask

Dataset_Path = r'F:\work\python\clone\dataset\rebar2d' # 数据集路径    
os.chdir(r"F:\work\dataset\rebar2D\train\TEMP")
for img_path in tqdm(os.listdir()):
    
    try:
    
        labelme_json_path = os.path.join(Dataset_Path, 'labelme_jsons', '.'.join(img_path.split('.')[:-1])+'.json')

        img_mask = labelme2mask_single_img(img_path, labelme_json_path)

        mask_path = img_path.split('.')[0] + '.png'

        # 将单通道灰度图像保存
        cv2.imwrite(os.path.join('../','mask',mask_path), img_mask)
    
    except Exception as E:
        print(img_path, '转换失败', E)
img_mask2=Image.open(r"F:\work\python\clone\dataset\rebar2d\mask\Image_20240622152022155.png")
print(img_mask2.size)

  5%|▍         | 15/301 [00:00<00:03, 72.86it/s]

Image_20240622152006003.png 转换失败 name 'class_info' is not defined
Image_20240622152022155.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622152022155.png 转换失败 name 'class_info' is not defined
Image_20240622152027051.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622152027051.png 转换失败 name 'class_info' is not defined
Image_20240622152032646.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622152032646.png 转换失败 name 'class_info' is not defined
Image_20240622152038023.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622152038023.png 转换失败 name 'class_info' is not defined
Image_20240622152043066.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622152043066.png 转换失败 name 'class_info' is not defined
Image_20240622152049220.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622152049220.png 转换失败 name 'class_info' is not defined
Image_20240622152054365.json 转换失败 'NoneType' object has no attribute 'shape'

 11%|█         | 33/301 [00:00<00:03, 79.77it/s]

Image_20240622152110857.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622152110857.png 转换失败 name 'class_info' is not defined
Image_20240622152117110.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622152117110.png 转换失败 name 'class_info' is not defined
Image_20240622152138391.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622152138391.png 转换失败 name 'class_info' is not defined
Image_20240622152201002.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622152201002.png 转换失败 name 'class_info' is not defined
Image_20240622152327307.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622152327307.png 转换失败 name 'class_info' is not defined
Image_20240622152413746.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622152413746.png 转换失败 name 'class_info' is not defined
Image_20240622152525162.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622152525162.png 转换失败 name 'class_info' is not defined

 16%|█▋        | 49/301 [00:00<00:03, 75.35it/s]

...

 92%|█████████▏| 277/301 [00:03<00:00, 78.40it/s]

Image_20240622155518451.png 转换失败 name 'class_info' is not defined
Image_20240622155627318.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622155627318.png 转换失败 name 'class_info' is not defined
Image_20240622155634193.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622155634193.png 转换失败 name 'class_info' is not defined
Image_20240622155638304.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622155638304.png 转换失败 name 'class_info' is not defined
Image_20240622155643472.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622155643472.png 转换失败 name 'class_info' is not defined
Image_20240622155648213.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622155648213.png 转换失败 name 'class_info' is not defined
Image_20240622155652944.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622155652944.png 转换失败 name 'class_info' is not defined
Image_20240622155658642.json 转换失败 'NoneType' object has no attribute 'shape'

100%|██████████| 301/301 [00:03<00:00, 81.24it/s]

Image_20240622155706238.png 转换失败 name 'class_info' is not defined
Image_20240622155713170.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622155713170.png 转换失败 name 'class_info' is not defined
Image_20240622155716771.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622155716771.png 转换失败 name 'class_info' is not defined
Image_20240622155719377.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622155719377.png 转换失败 name 'class_info' is not defined
Image_20240622155722311.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622155722311.png 转换失败 name 'class_info' is not defined
Image_20240622155724819.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622155724819.png 转换失败 name 'class_info' is not defined
Image_20240622155727835.json 转换失败 'NoneType' object has no attribute 'shape'
Image_20240622155727835.png 转换失败 name 'class_info' is not defined
Image_20240622155730477.json 转换失败 'NoneType' object has no attribute 'shape'

(3072, 2048)


In [27]:
img=labelme2mask_single_img(r"F:\work\python\clone\dataset\rebar2d\images\Image_20240622152006003.jpg", r"F:\work\python\clone\dataset\rebar2d\labelme_jsons\Image_20240622152006003.json")
cv2.imwrite(r"/utils/mask/mask.png", img)
img_mask=cv2.imread(r"/utils/mask/mask.png")
img_mask2=cv2.imread(r"F:\work\dataset\rebar2D\train\TEMP\Image_20240622152022155.png")

mask = img_mask[:, :, 0]
print(mask.shape)
cv2.imwrite(r"/utils/mask/mask.png", mask)
mask2=cv2.imread(r"/utils/mask/mask.png")
print(mask2.shape)
# print(img_mask2.shape)
# print(img.shape)

TypeError: 'NoneType' object is not subscriptable

In [25]:
from PIL import Image
import numpy as np
import os

def get_max_gray_value(image_path):
    # 打开图像
    img = Image.open(image_path).convert('L')  # 转换为灰度图像

    # 转换为numpy数组
    img_array = np.array(img)

    # 获取最高灰度值
    max_gray_value = img_array.max()
    print(f"最高灰度值: {max_gray_value}")

    return img_array, img

def replace_gray_value(img_array, old_value, new_value):
    # 将灰度值为old_value的像素转换为new_value
    img_array[img_array == old_value] = new_value

    # 转换为图像
    new_img = Image.fromarray(img_array)

    return new_img

if __name__ == "__main__":
    # 替换为您的图片路径
    # image_path = 'path_to_your_image.png'
    image_path = r"F:\work\python\clone\utils\mask"
    output_dir = r"F:\work\dataset\rebar2D\train\TEMP"
    for filename in os.listdir(image_path):
        if filename.endswith(".png"):
            input_path = os.path.join(image_path, filename)
            # 获取最高灰度值并返回图像数组
            img_array, img = get_max_gray_value(input_path)

            # 将灰度值为1的像素转换为255
            new_img = replace_gray_value(img_array, 1, 255)

            # 保存新图像
            new_image_path = os.path.join(output_dir, filename)
            new_img.save(new_image_path)
            print(f"新图像已保存到: {new_image_path}")
    # # 获取最高灰度值并返回图像数组
    # img_array, img = get_max_gray_value("/kaggle/input/rebar2d/train2/mask/Image_20240622152006003_mask.png")
    # 
    # # 将灰度值为1的像素转换为255
    # new_img = replace_gray_value(img_array, 1, 255)
    # 
    # # 保存新图像
    # new_image_path = 'path_to_save_new_image.png'
    # new_img.save(new_image_path)
    # print(f"新图像已保存到: {new_image_path}")


最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train\TEMP\mask.png


## labelme转mask-批量

In [20]:
os.chdir(Dataset_Path)
# os.mkdir('mask')
os.chdir('images')

In [21]:
print(tqdm(os.listdir()))

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]


In [6]:
Dataset_Path = r'F:\work\python\clone\dataset\rebar2d' # 数据集路径    
for img_path in tqdm(os.listdir()):
    
    try:
    
        labelme_json_path = os.path.join(Dataset_Path, 'labelme_jsons', '.'.join(img_path.split('.')[:-1])+'.json')

        img_mask = labelme2mask_single_img(img_path, labelme_json_path)

        mask_path = img_path.split('.')[0] + '.png'

        cv2.imwrite(os.path.join('../','mask',mask_path), img_mask)
    
    except Exception as E:
        print(img_path, '转换失败', E)
        

0it [00:00, ?it/s]


## 转换之后的mask保存在`masks`文件夹中

## 重命名和删除文件夹

In [18]:
os.chdir('../')
shutil.move('images', 'img_dir')
shutil.move('mask', 'ann_dir')
!rm -rf labelme_jsons
os.chdir('../')

## 得到最终的语义分割数据集

## 查看数据集目录结构

In [19]:
import seedir as sd
sd.seedir('Watermelon87_Semantic_Seg_Labelme', style='emoji', depthlimit=1)

📁 Watermelon87_Semantic_Seg_Labelme/
├─📁 img_dir/
└─📁 ann_dir/
